In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [2]:
run_200_1000 = pd.read_csv('data/20211026T184846_200-1000-iperf.csv')

In [7]:
def pckt_count(a):
    a = a.split(';')
    a = [int(i) for i in a[:-1]]
    return len(a)

run_200_1000['pck_ct'] = run_200_1000['packet_times'].apply(pckt_count)

run_200_1000['packet_size_total'] = run_200_1000['1->2Pkts'] + run_200_1000['2->1Pkts']

ratio_200_1000 = [0.2 for i in range(318)]

run_200_1000['ratio'] = ratio_200_1000

def agg_10sec(df):
    new_df = pd.DataFrame()
    min_time = df["Time"][0]
    ratio = df["ratio"][0]
    while min_time < df["Time"][len(df)-1]:
        temp_df = df[(df["Time"] >= min_time) & (df["Time"] < min_time+10)]
        row = temp_df[["1->2Bytes", "2->1Bytes", "1->2Pkts", "2->1Pkts", "packet_size_total", "pck_ct"]].sum().to_frame().T
        row["packet_sizes_var"] = temp_df["packet_size_total"].var()
        row["Time"] = min_time
        row["ratio"] = ratio
        new_df = new_df.append(row)
        min_time += 10
    return new_df.reset_index(drop=True)

agg_200_1000 = agg_10sec(run_200_1000)

agg_200_1000['latency'] = [200] * len(agg_200_1000)

In [10]:
full_df = pd.concat([agg_200_1000])

In [11]:
# Load the diabetes dataset
features = ['packet_size_total', 'pck_ct', 'packet_sizes_var', 'Time', 'ratio']
df_X = full_df[features]
df_y = full_df['latency']

# Use only one feature
#diabetes_X = diabetes_X[:, np.newaxis, 2]

# Split the data into training/testing sets
X_train, X_rem, y_train, y_rem = train_test_split(df_X, df_y, train_size=0.8, random_state=42)

X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, test_size=0.5)

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

# The coefficients
print("Coefficients: \n", regr.coef_)
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

Coefficients: 
 [0. 0. 0. 0. 0.]
Mean squared error: 0.00
Coefficient of determination: 1.00
